In [43]:
#config
resume = False
render = True
save_freq = 500
resume_pkl = "Q.pkl"
max_episodes = 100
test = False

In [1]:
import gym
# env = gym.make('ALE/Pong-v5', render_mode='rgb_array')
env = gym.make('PongDeterministic-v4', render_mode='rgb_array')

In [2]:
import cv2
import numpy as np
import random
import math
import time
import pickle

In [3]:
obv = env.reset()
obv[1]


{'lives': 0, 'episode_frame_number': 0, 'frame_number': 0}

In [4]:
observation = env.reset()
cumulated_reward = 0

# frames = []
# for t in range(1000):
# #     print(observation)
#     frames.append(env.render())
#     # very stupid agent, just makes a random action within the allowd action space
#     action = env.action_space.sample()
# #     print("Action: {}".format(t+1))
#     observation, reward, done, truncated, info = env.step(action)
# #     print(reward)
#     cumulated_reward += reward
#     if done:
#         print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
#         break
# print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))

# env.close()

In [48]:
# for frame in frames:
#     gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('frame', gray_frame)
#     cv2.waitKey(1)

In [49]:
# frames[200].shape

In [50]:
# greyframe = cv2.cvtColor(frames[300], cv2.COLOR_BGR2GRAY)

In [51]:
# greyframe

In [52]:
# np.unique(cv2.cvtColor(frames[200][34:], cv2.COLOR_BGR2GRAY))

In [53]:
# gray_frame = cv2.cvtColor(frames[-1][34:-16], cv2.COLOR_BGR2GRAY)
# #shape: (210-25=185 , 160)

# unique_values = [64, 123, 147, 236]
# coordinates = {}

# for value in unique_values:
#     coordinates[value] = np.where(gray_frame == value)


# # for value, (x, y) in coordinates.items():
# #     print(f"Value: {value}, Coordinates: {list(zip(x, y))}")
# #     print(f"Value: {value}, Coordinates: {len(list(zip(x, y)))}")
# #     print()
# # #bg
# print(f"Value: {64}, Coordinates: {(coordinates[64][0][0]+coordinates[64][0][-1])/2, (coordinates[64][1][0]+coordinates[64][1][-1])/2}")
# #left
# print(f"Value: {123}, Coordinates: {(coordinates[123][0][0]+coordinates[123][0][-1])/2, (coordinates[123][1][0]+coordinates[123][1][-1])/2}")
# #right
# print(f"Value: {147}, Coordinates: {(coordinates[147][0][0]+coordinates[147][0][-1])/2, (coordinates[147][1][0]+coordinates[147][1][-1])/2}")
# #ball
# print(f"Value: {236}, Coordinates: {(coordinates[236][0][0]+coordinates[236][0][-1])/2, (coordinates[236][1][0]+coordinates[236][1][-1])/2}")

In [5]:
env.action_space

Discrete(6)

In [55]:
# state = env.reset()
# done = False
# no_steps = 0
# while not done:
#     a = random.randint(0, 5)
#     state, reward, done,tun, info = env.step(a)
#     no_steps += 1

# print(no_steps)


In [56]:
# print(state.shape)

In [3]:
def discretizer(state):
    greyimg = cv2.cvtColor(state[34:-16], cv2.COLOR_BGR2GRAY)
    unique_values = [123, 147, 236]
    coordinates = {}
    for value in unique_values:
        coordinates[value] = np.where(greyimg == value)
    #left center
    #print(coordinates[123], coordinates[147], coordinates[236])
    if coordinates[123][0].any():
        lc = (coordinates[123][0][0]+coordinates[123][0][-1])/2
    else:
        lc = -1
    #right center
    if coordinates[147][0].any():
        rc = (coordinates[147][0][0]+coordinates[147][0][-1])/2
    else:
        rc = -1
    #ball center
    if coordinates[236][0].any():
        bc = ((coordinates[236][0][0]+coordinates[236][0][-1])/2, (coordinates[236][1][0]+coordinates[236][1][-1])/2)
    else:
        bc = (-1,-1)
    
    return int(lc), int(rc), int(bc[0]),int(bc[1])

In [58]:
# discretizer(state)

In [59]:
# frames[200].shape

In [60]:
# discretizer(frames[200])

In [61]:
Q = np.zeros([160//2, 160//2, 160//2, 160//2, 6],dtype=np.int8)


In [62]:
Q.nbytes
#31457280000 default dtype 
#3932160000 unit8 
#245760000 with//2

245760000

In [63]:
def policy(state:tuple):
    return np.argmax(Q[state])
def new_Q_value(reward: float, state_new: tuple, discount_factor=1):
    future_optimal_value = np.max(Q[state_new])
    learned_value = reward + discount_factor * future_optimal_value
    return learned_value

def learning_rate(n:int, min_rate=0.01):
    return max(min_rate, min(1.0, 1.0-math.log10((n+1)/25)))

def exploration_rate(n:int, min_rate=0.1):
    return max(min_rate, min(1, 1.0-math.log10((n+1)/25)))

In [64]:
print(env.reset()[0].shape)

(210, 160, 3)


In [7]:
from tqdm import tqdm

In [66]:
n_episodes = max_episodes

if not resume:
    for e in tqdm(range(n_episodes)):
        # print(*env.reset())
        # print(*env.reset[-1])
        current_state, done = discretizer(env.reset()[0]), False
        while done == False:
            action = policy(current_state)
            if np.random.random() < exploration_rate(e):
                action = env.action_space.sample()
            #print(env.step(action)[0].shape)
            obs, reward, done,tun, _ = env.step(action)
            # print("obs:",obs[0])
            new_state = discretizer(obs)
            
            lr = learning_rate(e)
            learnt_value = new_Q_value(reward, new_state)
            old_value = Q[current_state][action]
            Q[current_state][action] = (1-lr)*old_value + lr*learnt_value
            
            current_state = new_state

        # if e%save_freq == 0:
        #     with open("Q%d.pkl"%e,"wb") as f:
        #         pickle.dump(Q,f)

else:
    frames = []
    with open(resume_pkl,"rb") as f:
        Q = pickle.load(f)
    for e in tqdm(range(n_episodes)):
        # print(*env.reset())
        # print(*env.reset[-1])
        current_state, done = discretizer(env.reset()[0]), False
        while done == False:
            action = policy(current_state)
            if np.random.random() < exploration_rate(e):
                action = env.action_space.sample()
            #print(env.step(action)[0].shape)
            obs, reward, done,tun, _ = env.step(action)
            # print("obs:",obs[0])
            new_state = discretizer(obs)
            
            lr = learning_rate(e)
            learnt_value = new_Q_value(reward, new_state)
            old_value = Q[current_state][action]
            Q[current_state][action] = (1-lr)*old_value + lr*learnt_value
            
            current_state = new_state
            frames.append(obs)
        # if e%save_freq == 0:
        #     with open("Q%d.pkl"%e,"wb") as f:
        #         pickle.dump(Q,f)

env.close()

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\10536\AppData\Roaming\Python\Python38\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
100%|██████████| 100/100 [01:14<00:00,  1.34it/s]


In [ ]:
Q.nbytes

3932160000

In [ ]:

with open("Q.pkl", "wb") as f:
    pickle.dump(Q, f)


In [68]:
if test:
    frames = []
    env.reset()
    Q = pickle.load(open(resume_pkl, "rb"))
    # print(type(Q))
    current_state, done = discretizer(env.reset()[0]), False
    while done == False:
        action = policy(current_state)
        # print(action)
        #print(env.step(action)[0].shape)
        if np.random.random() < exploration_rate(100):
            action = env.action_space.sample()
        obs, reward, done,tun, _ = env.step(action)
        # print("obs:",obs[0])
        new_state = discretizer(obs)


        current_state = new_state
        frames.append(obs)
    for frame in frames:
        cv2.imshow('frame', frame)
        cv2.waitKey(1)

In [31]:
env.action_space.sample()

2

In [ ]:
if render:
    for frame in frames:
        cv2.imshow('frame', frame)
        cv2.waitKey(1)

KeyboardInterrupt: 

# Use deep learning

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm




class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(17, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.float()  # 将输入数据转换为float类型
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# 创建一个网络实例
net = Net()



In [16]:
from collections import deque
# 定义损失函数
criterion = nn.MSELoss()
# net = net.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
# 定义优化器
optimizer = optim.Adam(net.parameters(), lr=0.01)
def train(n_eposides):
    for e in tqdm(range(n_eposides)):
        frame_no = 0
        frames = deque(maxlen=16)
        env.reset()
        current_state, done = discretizer(env.reset()[0]), False
        while not done:
            if frame_no < 4:
                action = env.action_space.sample()
                obs, reward, done,_, __ = env.step(action)
                for i in range(4):
                    frames.append(current_state[i])
                frame_no += 1
                continue
            else:
                actions = [net(torch.tensor(list(frames+[i]))) for i in range(6)]
                action = actions.index(max(actions))
            for i in range(4):
                frames.append(torch.tensor(current_state[i])) 
            obs, reward, done, _,__ = env.step(action)
            current_state = discretizer(obs)
            optimizer.zero_grad()
            predicted_reward = net(torch.tensor(list(frames+[i])))
            loss = criterion(predicted_reward, torch.tensor([reward]))
            loss.backward()
            optimizer.step()
            frame_no += 1
            cv2.imshow('frame', np.asarray(obs))
train(20)


            



  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [ ]:
# 假设你已经有了状态信息和实际奖励
states = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
actual_reward = torch.tensor([10], dtype=torch.float32)

# 清除之前计算的梯度
optimizer.zero_grad()

# 步骤1：使用网络生成预测奖励
predicted_reward = net(states)

# 步骤2：计算预测奖励和实际奖励之间的损失
loss = criterion(predicted_reward, actual_reward)

# 步骤3：使用loss.backward()计算损失相对于模型参数的梯度
loss.backward()

# 步骤4：使用optimizer.step()更新模型参数
optimizer.step()
